# import library

In [1]:
import torch
import torchvision

import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from model.edsr import *
from model.vdsr import *
import numpy as np

In [2]:
import easydict
args = easydict.EasyDict({
         "n_threads": 8,
 
        "n_GPUs": 1,
 

        "seed": 1,
 
        "scale": (2,),
    "n_colors": 3,
    "act": "Relu",
    
    "n_resblocks": 32,
    "n_feats": 256,
    "shift_mean": True,
    'res_scale':0.1,
    
        "rgb_range": 255
 })




In [3]:
model=VDSR()

In [4]:
def test():
    model = EDSR(args)
    print("\nmodel.head[0].weight.data\n",model.head[0].weight.data)
    print('\nmodel.head[0].__class__\n',model.head[0].__class__)
    print('\nmodel.head[0].kernel_size\n',model.head[0].kernel_size)
    input = torch.autograd.Variable(torch.randn(32, 3, 224, 224))
    print('\ninput.size()\n',input.size())
    print('\nnp.prod(input.size()): ',np.prod(input.size()))
test()


model.head[0].weight.data
 tensor([[[[ 0.0318, -0.1106,  0.1914],
          [-0.0636, -0.1828, -0.1599],
          [-0.0373, -0.0097, -0.0080]],

         [[ 0.1088,  0.0551, -0.1558],
          [ 0.0453, -0.0801, -0.1581],
          [-0.0124,  0.1518,  0.0053]],

         [[ 0.0277, -0.0464, -0.0654],
          [-0.0764, -0.1893, -0.0944],
          [ 0.0394, -0.1519,  0.1823]]],


        [[[ 0.0580, -0.0132,  0.0599],
          [-0.1195,  0.0946,  0.1170],
          [-0.0987, -0.0500,  0.0701]],

         [[-0.0078, -0.0991, -0.1271],
          [-0.1648, -0.0592,  0.0261],
          [-0.0153,  0.0807, -0.1918]],

         [[-0.0067, -0.0908,  0.0882],
          [-0.1331,  0.0291, -0.0525],
          [-0.0527,  0.1638,  0.1906]]],


        [[[-0.1806, -0.1228, -0.1420],
          [ 0.1747, -0.1397,  0.0155],
          [ 0.1910, -0.0299, -0.0157]],

         [[ 0.1827, -0.0875,  0.1350],
          [-0.1424, -0.1198, -0.1252],
          [-0.0821, -0.0676,  0.0367]],

         [[-0.09

def VDSR_test():
    model = VDSR()
    print("\nmodel.head[0].weight.data\n",model.head[0].weight.data)
    print('\nmodel.head[0].__class__\n',model.head[0].__class__)
    print('\nmodel.head[0].kernel_size\n',model.head[0].kernel_size)
    input = torch.autograd.Variable(torch.randn(32, 3, 224, 224))
    print('\ninput.size()\n',input.size())
    print('\nnp.prod(input.size()): ',np.prod(input.size()))
VDSR_test()

In [5]:
def print_model_parm_nums(my_models=VDSR()):
    model = my_models#EDSR(args)
    trainable = filter(lambda x: x.requires_grad, model.parameters())
    num_params = sum([np.prod(p.size()) for p in filter(lambda x: x.requires_grad, model.parameters())])
    print('num of parameters: %.2fM ' % (int(num_params) / 1e6) +'\n')
    print('estimated of size %.2fM ' % (int(num_params) / 1e6*4) +'\n')
print_model_parm_nums()

num of parameters: 0.66M 

estimated of size 2.66M 



In [7]:
#from model.edsr import *
from model.srcnn import *
from model.espcn import *
from model.edsr_mobile import *
def print_model_parm_flops(my_models=models.resnet50()):
    multiply_adds = False
    list_conv=[]
    def conv_hook(self, input, output):
        Tt=lambda x:torch.tensor(x)
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
        kernel_ops = Tt(self.kernel_size[0]) * Tt(self.kernel_size[1]) * Tt(int(self.in_channels / self.groups))* Tt(2 if multiply_adds else 1)
        bias_ops = Tt(1 if self.bias is not None else 0)
        params = Tt(output_channels) * Tt(kernel_ops + bias_ops)
        flops = Tt(batch_size) * Tt(params) * Tt(output_height) * Tt(output_width)
        list_conv.append(flops)
    list_linear=[]
    def linear_hook(self, input, output):
        
        Tt=lambda x:torch.tensor(x)
        batch_size = Tt(input[0].size(0) if input[0].dim() == 2 else 1)

        weight_ops = Tt(self.weight.nelement()) * Tt(2 if multiply_adds else 1)
        bias_ops = Tt(self.bias.nelement())
        flops = Tt(batch_size) * Tt(weight_ops + bias_ops)
        list_linear.append(flops)
    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement())
        

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        Tt=lambda x:torch.tensor(x)
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
        kernel_ops =Tt(self.kernel_size) * Tt(self.kernel_size)
        bias_ops = 0
        params = Tt(output_channels) * Tt(kernel_ops + bias_ops)
        flops = Tt(batch_size) * Tt(params) * Tt(output_height) * Tt(output_width)
        list_pooling.append(flops)



            
    def register_hook(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            return
        for c in childrens:
                register_hook(c)

    resnet =my_models
     #EDSR(args)
    register_hook(resnet)
    with torch.no_grad():
        input = torch.rand(1,1,224,224)
        out = resnet(input)
    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling))
    print('Number of FLOPs: %.2fG' % (total_flops / 1e9)+'\n')
mo=ESPCN()#EDSR(args)
print_model_parm_flops(mo)
print_model_parm_nums(mo)

Number of FLOPs: 1.00G

num of parameters: 0.02M 

estimated of size 0.09M 

